In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchvision import datasets, transforms, models
from tqdm import tqdm
import time
from torch.utils.tensorboard import SummaryWriter



In [13]:
trainSet = pd.read_csv('Dataset/train.csv')
classes = trainSet[['id','species']].copy()
classes['id'] = classes['id'].astype(str)
classes['label'] = LabelEncoder().fit_transform(classes['species'])

image_folder = 'Dataset/images/'
imgs = []
labels = []
for i in sorted(os.listdir(image_folder)):
    id = i.split('.')[0]
    if id in classes['id'].values:
        labels.append(classes[classes['id'] == id]['label'].values[0])
        image = Image.open(os.path.join(image_folder, i)).convert('1')
        imgs.append(image)

print(f"There are {len(imgs)} images in the dataset")
print(f"There are {len(np.unique(labels))} labels in the dataset")

There are 990 images in the dataset
There are 99 labels in the dataset


In [14]:
X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.2, random_state=42, stratify=labels)
print(f"There are {len(X_train)} images in the training set")
print(f"There are {len(X_test)} images in the test set")
print(f"There are {len(np.unique(y_train))} classes in the training set")
print(f"There are {len(np.unique(y_test))} classes in the test set")

There are 792 images in the training set
There are 198 images in the test set
There are 99 classes in the training set
There are 99 classes in the test set


In [15]:
class CustomDataSet(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.transform(image=np.array(self.images[index],dtype=np.float32))['image']
        label = torch.tensor(self.labels[index], dtype=torch.long)
        return image, label


    
transform = A.Compose([
    A.Resize(64, 64),
    ToTensorV2()
])

trainDataSet = CustomDataSet(images=X_train, labels=y_train, transform=transform)
testDataSet = CustomDataSet(images=X_test, labels=y_test, transform=transform)
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')


In [16]:
class CNNmodel(nn.Module):
    def __init__(self):
        super(CNNmodel, self).__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1), # 64x64x1 -> 64x64x64
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2), # 64x64x64 -> 32x32x64

            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), # 32x32x64 -> 32x32x64
            nn.ReLU(),
            nn.BatchNorm2d(64), 
            nn.MaxPool2d(kernel_size=2, stride=2), # 32x32x64 -> 16x16x64

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), # 16x16x64 -> 16x16x128
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2), # 16x16x128 -> 8x8x128

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1), # 8x8x128 -> 8x8x256
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2, stride=2), # 8x8x256 -> 4x4x256

            nn.Flatten(), # 4x4x256 -> 4096
            nn.Linear(256*4*4, 256), # 4096 -> 256
            nn.ReLU(),
            nn.Linear(256, 99), # 256 -> 99
        ).to(DEVICE)



    def forward(self, x):
        x = self.cnn(x).to(DEVICE)
        x = F.softmax(x, dim=1).to(DEVICE)

        return x


In [17]:
batch_size = 4
trainDataLoader = torch.utils.data.DataLoader(trainDataSet, batch_size=batch_size)
testDataLoader = torch.utils.data.DataLoader(testDataSet, batch_size=batch_size)

min_loss_epoch = 0
min_loss_value = -1
best_model_weights_paths = {}

best_val_loss = float('inf')  # Initialize with a large value
best_epoch = -1
best_model_weights = None
train_losses = []
val_losses = []

criterion = nn.CrossEntropyLoss().to(DEVICE)
model = CNNmodel().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=3e-6)
num_epochs = 1000

writer = SummaryWriter('runs/cnnBN/'+time.strftime("%Y%m%d-%H%M%S"))

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(trainDataLoader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"):
        optimizer.zero_grad()
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(trainDataSet)
    train_losses.append(train_loss)
    
    # Validation loop
    model.eval()  
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        for images, labels in testDataLoader:
            labels = labels.to(DEVICE)
            images = images.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
    accuracy = correct_predictions / total_samples
    val_loss = 1-accuracy
    val_losses.append(val_loss)
    
    writer.add_scalar('Training Loss', train_loss, epoch)
    writer.add_scalar('Validation Loss', val_loss, epoch)

Epoch 1/1000:   0%|          | 0/198 [00:00<?, ?batch/s]

Epoch 659/1000:  90%|█████████ | 179/198 [00:03<00:00, 47.14batch/s]


KeyboardInterrupt: 

In [ ]:

best_epoch = val_losses.index(min(val_losses))
min_loss_epoch = best_epoch
min_loss_value = f'{min(val_losses):.4f}'
print(f"Min Train Loss: {min(train_losses)} at Epoch {train_losses.index(min(train_losses))}  Min Val Loss: {min_loss_value[criterion]} at Epoch {best_epoch}")


# Save the best model's weights
torch.save(model, f"best_model.pth")